In [1]:
import glob
import os
import shutil
import multiprocessing
import cv2
import numpy as np
from collections import Counter
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_pre
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_pre
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering, AgglomerativeClustering, OPTICS
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import normalized_mutual_info_score, silhouette_score
from matplotlib import pyplot as plt
%matplotlib inline

## Global

### Variables

In [4]:
SAMPLE_DIR = "data/traffic-small"
FINAL_DIR = "data/traffic"
dataset_dir = FINAL_DIR
is_sampling = dataset_dir == SAMPLE_DIR

# Sort is the key :/
dataset = sorted(glob.glob(dataset_dir + "/*.jpg"))

# Where we store the predictions
RESULT_DIR = "res"
# Where we store the extracted features
FEATURE_SAVING_DIR = "extracted_features"

pretrain_input_size = (54, 28)

N_CLUSTERS = 14

CPUS = multiprocessing.cpu_count()

### Utils

In [5]:
def load_data_as_lines(path):
    r""" Open text file by path and read all lines """
    with open(path, "r") as fh:
        lines = fh.readlines()
        
    # transform docs into lists of words
    raw_lines = [l.split() for l in lines]

    return raw_lines

def save_result_as_file(prediction, file_name="prediction.dat"):
    r""" Save the predicted result as a new file """
    file_content = "\n".join(list(map(str, prediction)))
    path = os.path.join(RESULT_DIR, file_name)
    with open(path, "w") as fd:
        fd.write(file_content) 

In [6]:
def save_formatted_name(prediction, model, net="res50", pca=False, random_id=None):
    fname = "%s_%s_%s%s%s.dat" % (
        model.__class__.__name__,
        net,
        "%sx%s" % pretrain_input_size,
        "_pca" if pca else "",
        "_rand%s" % random_id if random_id else ""
    )
    save_result_as_file(prediction, fname)
    print("Saved %s" % fname)

In [7]:
def shuffle(x):
    np.take(x, np.random.permutation(x.shape[0]), axis=0, out=x)

## Analytics
Analyze the data distribution first, so we can decide how to resize the image

In [ ]:
def analyze_data(dataset):
    dim_to_key = lambda dim: "({},{})={}".format(dim[0], dim[1], dim[0] * dim[1])
    
    dim_cnt = Counter()
    multiply_dim_cnt = Counter()

    images = []
    for path in dataset:
        img = cv2.imread(path)
        key = dim_to_key(img.shape)
        dim_cnt[key] += 1
        multiply_dim_cnt[img.shape[0] * img.shape[1]] += 1
        
    print(dim_cnt.most_common())
    print(multiply_dim_cnt.most_common())

## Preprocessing

### Image Preprocess

In [ ]:
def preprocess_img(dataset):
    images = []
    for path in dataset:
        img = image.load_img(path, target_size=pretrain_input_size)
        img = image.img_to_array(img)
    #     img = cv2.imread(path)
    #     img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    #     img = cv2.resize(img, pretrain_input_size)
    #     img = img / 255.0  # Normalization
        images.append(img)
        
    images = np.array(images)
    print(images.shape)
    
    return images
    

## Feature Extraction 

In [12]:
def init_feature_saving_dir():
    if not os.path.exists(FEATURE_SAVING_DIR) or \
        len(os.listdir(FEATURE_SAVING_DIR)) == 0:
        try:
            shutil.rmtree(FEATURE_SAVING_DIR)
        except:
            pass

        os.mkdir(FEATURE_SAVING_DIR)

In [13]:
SUPPORTED_MODELS = ["vgg", "resnet50"]

def save_batch_features(x,
                        model_name="vgg",
                        preprocessing=False,
                        pooling=None,
                        save_as_file=True,
                        filename=None):

    assert model_name in SUPPORTED_MODELS
    
    init_feature_saving_dir()
    input_dim = pretrain_input_size + (3,) # (25,25) -> (25,25,3)
    
    if model_name == "vgg":
        if preprocessing:
            x = vgg16_pre(x)
        model = VGG16(weights='imagenet',
                      include_top=False,
                      pooling=pooling)
        
    elif model_name == "resnet50":
        if preprocessing:
            x = resnet50_pre(x)
        model = ResNet50(weights='imagenet',
                         include_top=False,
                         pooling=pooling)

    y = model.predict(x,
                      workers=CPUS,
                      use_multiprocessing=True)
    
    print(y.shape)

    if save_as_file or filename:
        name = filename if filename else \
            "features-%s_%s%s%s%s" % (
                model_name,
                "%sx%s" % pretrain_input_size,
                "_%s" % pooling if pooling else "",
                "_%s" % "preprocess" if preprocessing else "",
                "_sample" if is_sampling else "_final",
            )
        np.save("%s/%s" % (
            FEATURE_SAVING_DIR, 
            name
        ), y)
        print("Saved as %s", name)

    return y

### Flatten

In [10]:
def flatten(x):
    return np.array([np.ndarray.flatten(img) for img in x])

## Dimensionality Reduction

### PCA

In [14]:
def get_pca_data(x_train, do_the_math=True, reduced_n_components=2400):
    def percvar(v):
        r"""Transform eigen/singular values into percents.
        Return: vector of percents, prefix vector of percents
        """
        # sort values
        s = np.sort(np.abs(v))
        # reverse sorting order
        s = s[::-1]
        # normalize
        s = s/np.sum(s)
        return s, np.cumsum(s)

    def perck(s, p):
        return next(i + 1 for i, v in enumerate(s) if v >= p)

    if do_the_math:
        X_std = StandardScaler().fit_transform(x_train)
        means = np.mean(X_std, axis=0)
        X_sm = X_std - means

        U,s,V = np.linalg.svd(X_sm)
        _, pv = percvar(s**2/(X_sm.shape[0]-1))

        percentage_explained = 99
        n_components = perck(pv, percentage_explained * 0.01)
    else:
        n_components = reduced_n_components
    
    print("Original: %d. After PCA: %d" % (x_train.shape[-1], n_components))

    svd = TruncatedSVD(n_components=n_components)
    return svd.fit_transform(x_train)

## Clustering

In [17]:
def generate_random_test_file(x, pca=False):
    r"""Let's not waste submissions and hope it's not too bad"""
    shuffle(x)
    random_state = np.random.randint(0, len(x) - 1)
    model = KMeans(n_clusters=N_CLUSTERS,
                   n_jobs=CPUS,
                   random_state=random_state)
    model.fit(x)
    prediction = model.labels_.tolist()
    save_formatted_name(prediction, model, pca=pca, random_id=random_state)

In [ ]:
def clustering(x, pca=False):
    # Generate random K-means to submit today
#     generate_random_test_file(x, pca=True)

#     model = AgglomerativeClustering(n_clusters=N_CLUSTERS, memory="tmp",
#                                  affinity="cosine", linkage="complete")
#     model = AgglomerativeClustering(n_clusters=N_CLUSTERS, memory="tmp",
#                                  affinity="cosine", linkage="average")
#     model = AgglomerativeClustering(n_clusters=N_CLUSTERS, memory="tmp",
#                                  affinity="euclidean", linkage="ward")
#     model = DBSCAN(eps=49.8, min_samples=3, n_jobs=CPUS)
#     model = KMeans(n_clusters=N_CLUSTERS, n_jobs=CPUS,
#                    random_state=np.random.randint(0, 100000))
#     model = OPTICS(min_samples=20, n_jobs=CPUS)
    model = SpectralClustering(n_clusters=N_CLUSTERS, random_state=4,
                           n_init=50, n_neighbors=10,
                          affinity="nearest_neighbors", n_jobs=CPUS)
    model.fit(x)
    prediction = model.labels_
    print(np.unique(prediction, return_counts=True))
    
    if is_sampling:
        y = load_data_as_lines(os.path.join(dataset_dir, "clusters.txt"))
        y = np.array(y).flatten().astype(int)
        nmi_score = normalized_mutual_info_score(y, prediction)
        print("NMI score: ", nmi_score)
        
    s_score = silhouette_score(x, prediction)
    print("Silhouette score: ", s_score)

    save_formatted_name(prediction, model, pca=pca)
    
    
    ##### Dim (54, 28)
    # RESNET50 + AgglomerativeClustering(elu, ward) 0.3769453717166426
    # RESNET50_PCA + AgglomerativeClustering(elu, ward) 0.3844810789874803
    # RESNET50_PCA + AgglomerativeClustering(cosine, complete) 0.36958888818960556

    # RESNET50 + KMEANS 0.36985722497762685
    # RESNET50_PCA + KMEANS 0.28

    # RESNET50 + DBSCAN(49.8, 3) 0.02377917875872419

    ##### Bad dimensions (64, 64)
    # VGG16_MAX + KMEANS 0.008307981394281364
    # VGG16_MAX_PCA + KMEANS 0.009747882300356864
    # VGG16 + KMEANS 0.009683774163696106
    # VGG16_PCA + KMEANS 0.010623286038995799
    # RESNET50 + KMEANS 0.009794626073636827
    # RESNET50_PCA + KMEANS 0.008068110910338717
    # RESNET50_MAX + KMEANS 0.01032285854898277
    # RESNET50_MAX_PCA + KMEANS 0.008569389462865706

    # VGG16 + DBSCAN(0.5, 5) 0.024794674645757197

In [13]:
def clustering_cure(x):
    r"""This doesn't work well with our data. Just keep it here as a ref."""
    cure_instance2 = cure(x, N_CLUSTERS,
                      number_represent_points=40,
                      compression=0.05)

    cure_instance2.process()
    
    clusters = cure_instance2.get_clusters()
    
    sample_size = 4209
    pred = [False for _ in range(sample_size)]

    label = 0
    for cluster in clusters:
        label += 1
        for i in cluster:
            pred[i] = label
            
    print(list(filter(lambda x: x > 1, pred))) # Count of non-dominating clusters
            
    y = load_data_as_lines(os.path.join(dataset_dir, "clusters.txt"))
    y = np.array(y).flatten().astype(int)
    nmi_score = normalized_mutual_info_score(y, prediction)
    print("NMI score: ", nmi_score)

In [ ]:
def main():
    # Optional
#     analyze_data(dataset)

    images = preprocess_img(dataset)
    
    features = save_batch_features(images, "resnet50")
    
    # Load extracted features locally
    x = features
    # Load final for processing
#     x = np.load(os.path.join(FEATURE_SAVING_DIR, "features-resnet50_54x28_final_7.npy"))
    # Load best random sample so far to evaluate metrics
#     x = np.load(os.path.join(FEATURE_SAVING_DIR, "resnet50_54x28_pca_s.npy"))
    
    x = flatten(x)
    
    # Optional PCA
#     x = get_pca_data(x)
#     np.save("%s/%s" % (FEATURE_SAVING_DIR, "resnet50_%sx%s_pca" % pretrain_input_size), x)
#     x = np.load(os.path.join(FEATURE_SAVING_DIR, "resnet50_54x28_pca.npy"))

# shuffle(x)
    
    clustering(x)

In [ ]:
main()